In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [91]:
inspections = pd.read_csv('inspections.csv')

In [92]:
inspections.head()

,activity_date,employee_id,facility_address,facility_city,facility_id,facility_name,facility_state,facility_zip,grade,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,serial_number,service_code,service_description
0,2017-05-09,EE0000593,17660 CHATSWORTH ST,GRANADA HILLS,FA0175397,HOVIK'S FAMOUS MEAT & DELI,CA,91344,A,OW0181955,JOHN'S FAMOUS MEAT & DELI INC.,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,HOVIK'S FAMOUS MEAT & DELI,ACTIVE,PR0168541,98,DAHDRUQZO,1,ROUTINE INSPECTION
1,2017-04-10,EE0000126,3615 PACIFIC COAST HWY,TORRANCE,FA0242138,SHAKEY'S PIZZA,CA,90505,A,OW0237843,"SCO, LLC",RESTAURANT (61-150) SEATS HIGH RISK,1638,SHAKEY'S PIZZA,ACTIVE,PR0190290,94,DAL3SBUE0,1,ROUTINE INSPECTION
2,2017-04-04,EE0000593,17515 CHATSWORTH ST,GRANADA HILLS,FA0007801,BAITH AL HALAL,CA,91344,A,OW0031150,SABIR MOHAMMAD SHAHID,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,BAITH AL HALAL,INACTIVE,PR0036723,95,DAL2PIKJU,1,ROUTINE INSPECTION
3,2017-08-15,EE0000971,44455 VALLEY CENTRAL WAY,LANCASTER,FA0013858,FOOD 4 LESS #306,CA,93536,A,OW0012108,"FOOD 4 LESS, INC.",RESTAURANT (0-30) SEATS HIGH RISK,1632,FOOD 4 LESS DELI/BAKERY#306,ACTIVE,PR0039905,98,DA0ZMAJXZ,1,ROUTINE INSPECTION
4,2016-09-26,EE0000145,11700 SOUTH ST,ARTESIA,FA0179671,PHO LITTLE SAIGON,CA,90701,A,OW0185167,PHO SOUTH ST INC,RESTAURANT (61-150) SEATS HIGH RISK,1638,PHO LITTLE SAIGON,ACTIVE,PR0173311,96,DA41DBXA2,1,ROUTINE INSPECTION


In [93]:
inspections.shape

(191371, 20)

In [101]:
inspections.facility_state[inspections.facility_state != 'CA'].shape

Series([], Name: facility_state, dtype: object)

In [97]:
inspections.facility_state[inspections.facility_state != 'CA'] = 'CA'

C:\Users\griz1\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
inspections.facility_state[inspections.facility_state == 'CA'].shape

(191371,)

In [40]:
inspections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191371 entries, 0 to 191370
Data columns (total 20 columns):
activity_date          191371 non-null object
employee_id            191371 non-null object
facility_address       191371 non-null object
facility_city          191371 non-null object
facility_id            191371 non-null object
facility_name          191371 non-null object
facility_state         191371 non-null object
facility_zip           191371 non-null object
grade                  191371 non-null object
owner_id               191371 non-null object
owner_name             191371 non-null object
pe_description         191371 non-null object
program_element_pe     191371 non-null object
program_name           190969 non-null object
program_status         191371 non-null object
record_id              191371 non-null object
score                  191371 non-null object
serial_number          191371 non-null object
service_code           191371 non-null object
service_descrip

In [78]:
inspections.describe()

,activity_date,employee_id,facility_address,facility_city,facility_id,facility_name,facility_state,facility_zip,grade,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,serial_number,service_code,service_description
count,191371,191371,191371,191371,191371,191371,191371,191371,191371,191371,191371,191371,191371,190969,191371,191371,191371,191371,191371,191371
unique,743,251,39282,182,44028,36625,1,3038,5,35711,34706,19,19,38979,3,47435,46,191318,3,3
top,2016-08-09,EE0000721,1000 VIN SCULLY AVE,LOS ANGELES,FA0019271,SUBWAY,CA,90012,A,OW0029458,RALPHS GROCERY COMPANY,RESTAURANT (0-30) SEATS HIGH RISK,1632,SUBWAY,ACTIVE,CA,90,CA,1,ROUTINE INSPECTION
freq,562,4006,383,53669,380,576,191371,2111,181226,2093,2093,38869,38869,630,169423,54,24563,54,188410,188410


In [102]:
np.unique(inspections['score'])

array([ 54,  55,  57,  58,  59,  60,  62,  63,  64,  65,  66,  67,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,
        95,  96,  97,  98,  99, 100], dtype=int64)